## Modules

In [6]:
import os
import sys
sys.path.append('..')

import numpy as np
import pandas as pd

from datetime import datetime, timedelta
from itertools import product
from pandas import ExcelWriter
from scipy.stats import norm

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# `bsoption` module
from bsoption.bsmodel import *

## Exponential underlying price projection & option price

### Function

In [4]:
def getexpsim(spot, goal, optype, strike, sig, expiry, startdate, enddate):
    """Obtain projected option price curve based on stable exponential growth model."""
    # Expiry period
    numdays = (enddate - startdate).days
    # Daily growth rate
    daypct = np.log(goal / spot) / numdays
    # Option price dataframe structure
    cols = ['tradedate', 'spot', 'optype', 'tdays', 'sig', 'opprice', 'delta', 'theta', 'vega']
    dfprice = pd.DataFrame(columns=cols)
    # Compute each day's option price and greeks:
    for num in range(numdays + 1):
        date = startdate + timedelta(days=num)
        price = spot * np.exp(num * daypct)   # projected price 
        tdays = (expiry - date).days  # number of days to expiry
        BSop = BSModel(price, strike, tdays, sig / 100) # BSModel object
        opprice = BSop.getopprice(optype) # option price
        delta = BSop.getdelta(optype) # delta
        theta = BSop.gettheta(optype) # theta
        vega = BSop.vega # vega
        rowdict = {'tradedate': date, 'spot': round(price, 2), 'optype': optype, 'tdays': tdays, 'sig': sig, 
                   'opprice': round(opprice, 2), 'delta': round(delta, 4),
                   'theta': round(theta, 2), 'vega': round(vega, 2)}
        dfprice = dfprice.append(rowdict, ignore_index=True)
    
    dfprice = dfprice.set_index('tradedate')        
    
    return dfprice
    
    

### Example 1: ETH call

In [7]:
startdt1 = datetime(2022, 3, 20)
enddt1 = datetime(2022, 4, 29)
expiry1 = enddt1
spot1 = 2800
goal1 = 4000
strike1 = 3600
sig1 = 64
optype1 = 'C'

dfopt1 = getexpsim(spot1, goal1, optype1, strike1, sig1, expiry1, startdt1, enddt1)
dfopt1

..\bsoption\bsmodel.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  d_1 = (np.log(self.S / self.K) + self.T * (self.rf + 0.5 * (self.sig ** 2))) / (self.sig * (self.T ** 0.5))
..\bsoption\bsmodel.py:62: RuntimeWarning: invalid value encountered in double_scalars
  t_1 = - self.S * norm.pdf(self.d_1) * self.sig / (2 * (self.T) ** 0.5)
..\bsoption\bsmodel.py:65: RuntimeWarning: invalid value encountered in double_scalars
  t_1 = - self.S * norm.pdf(self.d_1) * self.sig / (2 * (self.T) ** 0.5)
..\bsoption\bsmodel.py:31: RuntimeWarning: invalid value encountered in double_scalars
  self.gamma = norm.pdf(self.d_1) / (self.S * self.sig * (self.T ** 0.5))


,spot,optype,tdays,sig,opprice,delta,theta,vega
tradedate,,,,,,,,
2022-03-20,2800.00,C,40,64,38.67,0.1400,-1.65,2.06
2022-03-21,2825.08,C,39,64,40.57,0.1459,-1.73,2.11
2022-03-22,2850.38,C,38,64,42.56,0.1521,-1.82,2.16
2022-03-23,2875.91,C,37,64,44.66,0.1586,-1.92,2.22
2022-03-24,2901.67,C,36,64,46.86,0.1654,-2.01,2.27
2022-03-25,2927.66,C,35,64,49.19,0.1726,-2.12,2.32
2022-03-26,2953.88,C,34,64,51.64,0.1801,-2.23,2.37
2022-03-27,2980.34,C,33,64,54.22,0.1880,-2.34,2.42
2022-03-28,3007.03,C,32,64,56.95,0.1963,-2.46,2.46


### Example 2: HHI Put